In [ ]:
#  Load the 102 Flowers dataset using TensorFlow
import tensorflow_datasets as tfds
flowers102, info = tfds.load('oxford_flowers102', split='train', with_info=True, as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

...

Generating test examples...:   0%|          | 0/6149 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.I6ZESC_2.1.1/oxford_flowers102-test.tfrecord*…

Generating validation examples...:   0%|          | 0/1020 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/oxford_flowers102/incomplete.I6ZESC_2.1.1/oxford_flowers102-validation.tfr…

Dataset oxford_flowers102 downloaded and prepared to /root/tensorflow_datasets/oxford_flowers102/2.1.1. Subsequent calls will reuse this data.


In [ ]:
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-_qsktskd
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-_qsktskd
  Resolved https://github.com/openai/CLIP.git to commit dcba3cb2e2827b402d2701e7e1c7d9fed8a20ef1
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.4/54.4 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_

In [ ]:
import os
import clip
import torch

In [ ]:
# Load the model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

100%|███████████████████████████████████████| 338M/338M [00:03<00:00, 91.4MiB/s]


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
# Access a specific data sample
#for image, label in flowers102.take(1):
 #   print(f"Image shape: {image.shape}")
  #  print(f"Label: {label}")

In [ ]:
# 2. Prepare the inputs
from PIL import Image
image, class_id = next(iter(flowers102.take(1)))
image = Image.fromarray(image.numpy().astype('uint8'), 'RGB')
image_input = preprocess(image).unsqueeze(0).to(device)
text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}") for c in info.features['label'].names]).to(device)

In [ ]:
# Calculate features
with torch.no_grad():
    image_features = model.encode_image(image_input)
    text_features = model.encode_text(text_inputs)

In [ ]:
# Pick the top 5 most similar labels for the image
image_features /= image_features.norm(dim=-1, keepdim=True)
text_features /= text_features.norm(dim=-1, keepdim=True)
similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
values, indices = similarity[0].topk(5)

In [ ]:
# 4. Print the top predictions
print("\nTop predictions:\n")
for value, index in zip(values.squeeze(), indices.squeeze()):
    print(f"{info.features['label'].names[index]:>16s}: {100 * value.item():.2f}%")


Top predictions:

      water lily: 57.31%
           lotus: 27.27%
      sword lily: 6.35%
      siam tulip: 2.26%
        bromelia: 1.40%


In [ ]:
import os
import clip
import torch
from torchvision.datasets import Flowers102
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from tqdm import tqdm
from torchvision.transforms import ToTensor

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device)

In [ ]:
# Load the dataset
root = os.path.expanduser("~/.cache")
# Load the dataset (use 'split="train"' for training data)
train_dataset = Flowers102(root, split="train", transform=preprocess, download=True)
test_dataset = Flowers102(root, split="test", transform=preprocess, download=True)

100%|██████████| 344862509/344862509 [00:09<00:00, 35063648.14it/s]


Extracting /root/.cache/flowers-102/102flowers.tgz to /root/.cache/flowers-102


100%|██████████| 502/502 [00:00<00:00, 406318.14it/s]


100%|██████████| 14989/14989 [00:00<00:00, 10084764.62it/s]


In [ ]:
def get_features(dataset):
    all_features = []
    all_labels = []

    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=100)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

In [ ]:
# Calculate the image features
train_features, train_labels = get_features(train_dataset)
test_features, test_labels = get_features(test_dataset)


100%|██████████| 62/62 [19:36<00:00, 18.97s/it]


In [ ]:
# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

LogisticRegression(C=0.316, max_iter=1000, random_state=0, verbose=1)

In [ ]:
# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

Accuracy = 93.283


i used different promptr for top 5 the lowest accuracy class and added some explaination about flowers for better accuracy - question 7 - ت

In [ ]:
low_accuracy_classes = sorted(class_accuracy.items(), key=lambda x: x[1] / class_counts[x[0]], reverse=False)[:5]

# Print the top 5 classes with the lowest accuracy
print("\nTop 5 Classes with Lowest Accuracy:")
for label, accuracy in low_accuracy_classes:
    accuracy_percentage = (accuracy / class_counts[label]) * 100
    print(f"{label:>16s}: {accuracy_percentage:.3f}%")


Top 5 Classes with Lowest Accuracy:
canterbury bells: 40.000%
         petunia: 61.765%
      sword lily: 65.455%
       sweet pea: 75.000%
      canna lily: 75.806%


In [ ]:
feature_descriptions = {
    "canterbury bells": "bell-shaped purple flowers and long, slender stems",
    "petunia": "colorful, funnel-shaped flowers and compact growth habit",
    "sword lily": "long, sword-like leaves and vibrant flower spikes",
    "sweet pea": "delicate, fragrant flowers and climbing vines",
    "canna lily": "large, tropical-looking leaves and showy flower heads",
    "pink primrose": "",
}

text_inputs = torch.cat([clip.tokenize(f"a photo of a {c}, with its distinctive {feature_descriptions.get(c, '')}") for c in info.features['label'].names]).to(device)

In [ ]:
# logistic regression  the updated prompt
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

#  logistic regression classifier
predictions = classifier.predict(test_features)

#  class-wise accuracy
class_accuracy = {label: 0 for label in info.features['label'].names}
class_counts = {label: 0 for label in info.features['label'].names}

for true_label, pred_label in zip(test_labels, predictions):
    class_counts[info.features['label'].names[true_label]] += 1
    if true_label == pred_label:
        class_accuracy[info.features['label'].names[true_label]] += 1

#  overall accuracy
overall_accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
print(f"Overall Accuracy = {overall_accuracy:.3f}")

#  the top 5 classes  the lowest accuracy
low_accuracy_classes = sorted(class_accuracy.items(), key=lambda x: x[1] / class_counts[x[0]], reverse=False)[:5]
print("\nTop 5 Classes with Lowest Accuracy:")
for label, accuracy in low_accuracy_classes:
    accuracy_percentage = (accuracy / class_counts[label]) * 100
    print(f"{label:>16s}: {accuracy_percentage:.3f}%")

Overall Accuracy = 93.283

Top 5 Classes with Lowest Accuracy:
canterbury bells: 40.000%
         petunia: 61.765%
      sword lily: 65.455%
       sweet pea: 75.000%
      canna lily: 75.806%


**the output of change and update prompt still the same az before so now:
i wanna try Data Augmentation**

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import torch

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.Resizing(224, 224),  # Resize the images to 224x224
    tf.keras.layers.RandomFlip("horizontal_and_vertical"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.2),
])

In [ ]:
# Create a new dataset with data augmentation
train_dataset_aug = flowers102.map(lambda x, y: (data_augmentation(x, training=True), y))
train_dataset_aug = train_dataset_aug.take(train_size)
train_dataset_aug = train_dataset_aug.batch(100)

In [ ]:
train_features_aug = []
train_labels_aug = []

In [ ]:
for batch in tqdm(train_dataset_aug, total=len(train_dataset_aug)):
    images, labels = batch
    images = torch.from_numpy(images.numpy()).permute(0, 3, 1, 2).to(device)  # Reshape the images to match CLIP's input
    with torch.no_grad():
        image_features = model.encode_image(images)
    train_features_aug.append(image_features.cpu().numpy())
    train_labels_aug.append(labels.numpy())

100%|██████████| 11/11 [03:22<00:00, 18.37s/it]


In [ ]:
train_features_aug = np.concatenate(train_features_aug)
train_labels_aug = np.concatenate(train_labels_aug)

In [ ]:
# Perform logistic regression with the augmented dataset
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features_aug, train_labels_aug)

LogisticRegression(C=0.316, max_iter=1000, random_state=0, verbose=1)

In [ ]:
predictions = classifier.predict(test_features)

In [ ]:
# Calculate class-wise accuracy
class_accuracy = {label: 0 for label in info.features['label'].names}
class_counts = {label: 0 for label in info.features['label'].names}

In [ ]:
for true_label, pred_label in zip(test_labels, predictions):
    class_counts[info.features['label'].names[true_label]] += 1
    if true_label == pred_label:
        class_accuracy[info.features['label'].names[true_label]] += 1

In [ ]:
# Calculate overall accuracy
overall_accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
print(f"Overall Accuracy = {overall_accuracy:.3f}")

# Print the top 5 classes with the lowest accuracy
low_accuracy_classes = sorted(class_accuracy.items(), key=lambda x: x[1] / class_counts[x[0]], reverse=False)[:5]
print("\nTop 5 Classes with Lowest Accuracy:")
for label, accuracy in low_accuracy_classes:
    accuracy_percentage = (accuracy / class_counts[label]) * 100
    print(f"{label:>16s}: {accuracy_percentage:.3f}%")

Overall Accuracy = 1.350

Top 5 Classes with Lowest Accuracy:
   pink primrose: 0.000%
hard-leaved pocket orchid: 0.000%
english marigold: 0.000%
      tiger lily: 0.000%
     moon orchid: 0.000%


**lets try Class Weighting**

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

In [ ]:
# Calculate the class weights
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(train_labels), y=train_labels)
class_weight_dict = dict(zip(np.unique(train_labels), class_weights))

In [ ]:
# Perform logistic regression with class weighting
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1, class_weight=class_weight_dict)
classifier.fit(train_features, train_labels)

LogisticRegression(C=0.316,
                   class_weight={0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0, 4: 1.0, 5: 1.0,
                                 6: 1.0, 7: 1.0, 8: 1.0, 9: 1.0, 10: 1.0,
                                 11: 1.0, 12: 1.0, 13: 1.0, 14: 1.0, 15: 1.0,
                                 16: 1.0, 17: 1.0, 18: 1.0, 19: 1.0, 20: 1.0,
                                 21: 1.0, 22: 1.0, 23: 1.0, 24: 1.0, 25: 1.0,
                                 26: 1.0, 27: 1.0, 28: 1.0, 29: 1.0, ...},
                   max_iter=1000, random_state=0, verbose=1)

In [ ]:
# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)

In [ ]:
# Calculate class-wise accuracy
class_accuracy = {label: 0 for label in info.features['label'].names}
class_counts = {label: 0 for label in info.features['label'].names}

for true_label, pred_label in zip(test_labels, predictions):
    class_counts[info.features['label'].names[true_label]] += 1
    if true_label == pred_label:
        class_accuracy[info.features['label'].names[true_label]] += 1

In [ ]:
# Calculate overall accuracy
overall_accuracy = np.mean((test_labels == predictions).astype(float)) * 100.
print(f"Overall Accuracy = {overall_accuracy:.3f}")

Overall Accuracy = 93.283


In [ ]:
# Print the top 5 classes with the lowest accuracy
low_accuracy_classes = sorted(class_accuracy.items(), key=lambda x: x[1] / class_counts[x[0]], reverse=False)[:5]
print("\nTop 5 Classes with Lowest Accuracy:")
for label, accuracy in low_accuracy_classes:
    accuracy_percentage = (accuracy / class_counts[label]) * 100
    print(f"{label:>16s}: {accuracy_percentage:.3f}%")


Top 5 Classes with Lowest Accuracy:
canterbury bells: 40.000%
         petunia: 61.765%
      sword lily: 65.455%
       sweet pea: 75.000%
      canna lily: 75.806%
